In [6]:
#%pip install --user pandas
#%pip install --user graphviz
#%pip install --user torch
#%pip install --user transformers
#%pip install --user nltk
#%pip install --user openai
#%pip install --user python-dotenv


  Using cached https://files.pythonhosted.org/packages/64/62/f19d1e9023aacb47241de3ab5a5d5fedf32c78a71a9e365bb2153378c141/python_dotenv-0.21.1-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [13]:

import pandas as pd
import openai
import torch
from openai import OpenAI
import os
from dotenv import load_dotenv
from abc import ABC, abstractmethod
from typing import Dict

In [17]:
# Load environment variables from the .env file
load_dotenv()
# Get the OpenAI API key from the environment variables
client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv('GPT_KEY')
)

In [43]:
class LLMEndpointBase(ABC):

    @abstractmethod
    def ask(self, prompt_dict: Dict[str, str]) -> str:
        """
        Ask the LLM endpoint something

        Arguments:
            prompt_dict: A dictionary with the following keys:
                systemRole: The system prompt for this user
                user: The current user
                context: The previous context
                message: The message to the LLM agent
                model: The designated model to be used.
        
        Returns:
            response: response from LLM agent
        """
        pass

class ChatGPTEndpoint(LLMEndpointBase):

    def ask(self, prompt_dict: Dict[str, str]) -> str:
        try:
            # Create a prompt from the prompt_dict
            inputmessages = [
                {"role": "system", "content": prompt_dict['systemRole']},
                {"role": "user", "content": f"{prompt_dict['user']} {prompt_dict['context']} {prompt_dict['message']}"}
            ]

            # Make a request to the OpenAI API
            response = client.chat.completions.create(
                model="gpt-3.5-turbo",  # Specify the model you want to use
                messages=inputmessages,
                max_tokens=150
            )
            return response.choices[0].message
        except Exception as e:
            print(f"An error occurred: {e}")
            return None

# Example usage
if __name__ == "__main__":
    # Define the prompt_dict according to the required structure
    prompt_dict = {
        "systemRole": "Explain concepts in simple terms",
        "user": "Test User",
        "context": "Previous conversation context if any",
        "message": "Explain the concept of artificial intelligence."
        ""
    }
    
    # Create an instance of ChatGPTEndpoint
    endpoint = ChatGPTEndpoint()
    
    # Get the response from the ChatGPT endpoint
    response = endpoint.ask(prompt_dict)
    
    if response:
        print("ChatGPT Response:", response)
    else:
        print("Failed to get response from ChatGPT.")

ChatGPT Response: ChatCompletionMessage(content='Artificial Intelligence (AI) is a field of computer science that involves creating machines or systems that can perform tasks that typically require human intelligence. These tasks include things like learning, problem-solving, understanding natural language, and making decisions. AI systems learn from data, identify patterns, and make decisions or predictions based on that information. Examples of AI in everyday life include virtual assistants like Siri or Alexa, recommendation systems on streaming platforms, and autonomous vehicles.', role='assistant', function_call=None, tool_calls=None)
